In [20]:
import math
import os
import random
import sys
# import h5py
import logging
import numpy as np
from utils.data_utils import *
from models.motionpredictor import *
import torch
import torch.optim as optim
import argparse
import matplotlib.pyplot as plt

In [21]:
args = {
    "learning_rate": 0.00001,
    "learning_rate_decay_factor": 0.95,
    "learning_rate_step": 10000,
    "batch_size": 128,
    "iterations": int(2e4),
    "test_every": 100,
    "size": 512,
    "seq_length_in": 50,
    "seq_length_out": 10,
    # Directories
    "data_dir": os.path.normpath("../data/h3.6m/dataset"),
    "train_dir": os.path.normpath("./experiments/"),
    "action": 'all',
    "log_level": 20,
    "log_file": ''
}

In [22]:
train_dir = os.path.normpath(os.path.join( args['train_dir'], args['action'],
	'out_{0}'.format(args['seq_length_out']),
	'iterations_{0}'.format(args['iterations']),
	'size_{0}'.format(args['size']),
	'lr_{0}'.format(args['learning_rate'])))

In [23]:
args['seq_length_out'] # Number of frames that the decoder has to predict

10

In [24]:
# Logging
if args['log_file']=='':
	logging.basicConfig(format='%(levelname)s: %(message)s',level=args['log_level'])
else:
	logging.basicConfig(filename=args['log_file'],format='%(levelname)s: %(message)s',level=args['log_level'])

In [25]:
# Detect device
if torch.cuda.is_available():
	logging.info(torch.cuda.get_device_name(torch.cuda.current_device()))
else:
	logging.info("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

INFO: NVIDIA GeForce RTX 3090


In [26]:
device

device(type='cuda')

In [27]:
logging.info("Train dir: "+train_dir)
os.makedirs(train_dir, exist_ok=True)

INFO: Train dir: experiments/all/out_10/iterations_20000/size_512/lr_1e-05


In [28]:
"""Train a seq2seq model on human motion"""
# Set of actions
actions           = define_actions( args['action'] ) # like "walking", "eating", "smoking", "discussion",  "directions", etc
number_of_actions = len( actions )

# Loads data for training/testing and normalizes it
# seq_length_in: number of frames to use in the burn-in sequence
# seq_length_out: number of frames to use in the output sequence
train_set, test_set, data_mean, data_std, dim_to_ignore, dim_to_use = read_all_data(actions, args['seq_length_in'], args['seq_length_out'], args['data_dir'])

INFO: Reading training data (seq_len_in: 50, seq_len_out 10).
INFO: Reading subject 1, action walking, subaction 1
INFO: Reading subject 1, action walking, subaction 2
INFO: Reading subject 1, action eating, subaction 1
INFO: Reading subject 1, action eating, subaction 2
INFO: Reading subject 1, action smoking, subaction 1
INFO: Reading subject 1, action smoking, subaction 2
INFO: Reading subject 1, action discussion, subaction 1
INFO: Reading subject 1, action discussion, subaction 2
INFO: Reading subject 1, action directions, subaction 1
INFO: Reading subject 1, action directions, subaction 2
INFO: Reading subject 1, action greeting, subaction 1
INFO: Reading subject 1, action greeting, subaction 2
INFO: Reading subject 1, action phoning, subaction 1
INFO: Reading subject 1, action phoning, subaction 2
INFO: Reading subject 1, action posing, subaction 1
INFO: Reading subject 1, action posing, subaction 2
INFO: Reading subject 1, action purchases, subaction 1
INFO: Reading subject 1, 

In [29]:
# Create model for training only
model = MotionPredictor(args['seq_length_in'], args['seq_length_out'],
    args['size'], # hidden layer size
    args['batch_size'], args['learning_rate'],
    args['learning_rate_decay_factor'],
    len( actions ))
model = model.to(device)

INFO: Input size is 69


In [30]:
model

MotionPredictor(
  (cell): GRUCell(69, 512)
  (fc1): Linear(in_features=512, out_features=69, bias=True)
  (fc_mean): Linear(in_features=512, out_features=512, bias=True)
  (fc_var): Linear(in_features=512, out_features=512, bias=True)
)

In [31]:
# This is the training loop
loss, val_loss = 0.0, 0.0
current_step   = 0
all_losses     = []
all_val_losses = []

# The optimizer
#optimiser = optim.SGD(model.parameters(), lr=args.learning_rate)
optimiser = optim.Adam(model.parameters(), lr=args['learning_rate'], betas = (0.9, 0.999))

In [32]:
for _ in range(args['iterations']):
	optimiser.zero_grad()
	# Set a flag to compute gradients
	model.train()
	# === Training step ===

	# Get batch from the training set
	encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch(train_set,actions,device)

	# Forward pass
	preds, mu, logvar = model(encoder_inputs, decoder_inputs,device)

	# Loss: Mean Squared Errors
	rec_loss = (preds-decoder_outputs)**2
	kld_loss = -0.5 * (1 + logvar - mu**2 - torch.exp(logvar))
	step_loss = torch.mean(torch.sum(kld_loss, axis=1)) + rec_loss.mean()
	val_loss  = step_loss.mean()

	# step_loss = (preds-decoder_outputs)**2
	# step_loss = step_loss.mean()

	# Backpropagation
	step_loss.backward()
	# Gradient descent step
	optimiser.step()

	step_loss = step_loss.cpu().data.numpy()

	if current_step % 10 == 0:
		logging.info("step {0:04d}; step_loss: {1:.4f}".format(current_step, step_loss ))
	loss += step_loss / args['test_every']
	current_step += 1
	# === step decay ===
	if current_step % args['learning_rate_step'] == 0:
		args['learning_rate'] = args['learning_rate']*args['learning_rate_decay_factor']
		optimiser = optim.Adam(model.parameters(),lr=args['learning_rate'], betas = (0.9, 0.999))
		print("Decay learning rate. New value at " + str(args['learning_rate']))

	# Once in a while, save checkpoint, print statistics.
	if current_step % args['test_every'] == 0:
		model.eval()
		# === Validation ===
		encoder_inputs, decoder_inputs, decoder_outputs = model.get_batch(test_set,actions,device)
		preds, mu, logvar = model(encoder_inputs, decoder_inputs, device)

		step_loss = (preds-decoder_outputs)**2
		# kld_loss =-0.5 * (1 + logvar - mu**2 - torch.exp(logvar))
		# step_loss = torch.mean(torch.sum(kld_loss, axis=1)) + rec_loss
		val_loss  = step_loss.mean()

		print("\n============================\n"
			"Global step:         %d\n"
			"Learning rate:       %.4f\n"
			"Train loss avg:      %.4f\n"
			"--------------------------\n"
			"Val loss:            %.4f\n"
			"============================\n" % (current_step,
			args['learning_rate'], loss,val_loss))
		all_val_losses.append([current_step,val_loss.cpu().detach().numpy()])
		all_losses.append([current_step,loss])
		torch.save(model, train_dir + '/model_' + str(current_step))
		# Reset loss
		loss = 0

INFO: step 0000; step_loss: 9.4195
INFO: step 0010; step_loss: 8.6104
INFO: step 0020; step_loss: 8.3881
INFO: step 0030; step_loss: 8.7029
INFO: step 0040; step_loss: 7.9822
INFO: step 0050; step_loss: 7.8170
INFO: step 0060; step_loss: 8.0440
INFO: step 0070; step_loss: 7.3383
INFO: step 0080; step_loss: 7.4395
INFO: step 0090; step_loss: 7.4545
INFO: step 0100; step_loss: 7.2152



Global step:         100
Learning rate:       0.0000
Train loss avg:      8.0931
--------------------------
Val loss:            1.0328



INFO: step 0110; step_loss: 7.4384
INFO: step 0120; step_loss: 7.0064
INFO: step 0130; step_loss: 7.1717
INFO: step 0140; step_loss: 7.4223
INFO: step 0150; step_loss: 6.5549
INFO: step 0160; step_loss: 6.3093
INFO: step 0170; step_loss: 6.9458
INFO: step 0180; step_loss: 7.0900
INFO: step 0190; step_loss: 5.9221
INFO: step 0200; step_loss: 6.0044



Global step:         200
Learning rate:       0.0000
Train loss avg:      6.8519
--------------------------
Val loss:            0.8727



INFO: step 0210; step_loss: 6.0687
INFO: step 0220; step_loss: 5.9591
INFO: step 0230; step_loss: 5.2471
INFO: step 0240; step_loss: 6.3810
INFO: step 0250; step_loss: 5.3368
INFO: step 0260; step_loss: 5.5920
INFO: step 0270; step_loss: 4.9957
INFO: step 0280; step_loss: 5.3411
INFO: step 0290; step_loss: 5.7792
INFO: step 0300; step_loss: 5.0582


...


Global step:         2300
Learning rate:       0.0000
Train loss avg:      1.2766
--------------------------
Val loss:            0.3199



INFO: step 2310; step_loss: 1.2283
INFO: step 2320; step_loss: 1.2634
INFO: step 2330; step_loss: 1.2661
INFO: step 2340; step_loss: 1.2471
INFO: step 2350; step_loss: 1.1994
INFO: step 2360; step_loss: 1.1782
INFO: step 2370; step_loss: 1.1643
INFO: step 2380; step_loss: 1.2217
INFO: step 2390; step_loss: 1.2091
INFO: step 2400; step_loss: 1.2126



Global step:         2400
Learning rate:       0.0000
Train loss avg:      1.2209
--------------------------
Val loss:            0.3323



INFO: step 2410; step_loss: 1.1954
INFO: step 2420; step_loss: 1.1767
INFO: step 2430; step_loss: 1.1384


KeyboardInterrupt: 

In [ ]:
vlosses = np.array(all_val_losses)
tlosses = np.array(all_losses)

In [15]:
# Plot losses
plt.plot(vlosses[:,0],vlosses[:,1],'b')
plt.plot(tlosses[:,0],tlosses[:,1],'r')
plt.show()

In [16]:
np.save("CVAE01.npy", vlosses)

In [17]:
n_params = 0
for p in model.parameters():
    n_params += np.prod(p.shape)

print("Parámeters: {0:,}".format(n_params)) 

Parámeters: 1,456,197


In [33]:
bl = np.load("CVAE01.npy")

In [34]:
np.min(bl)

0.11384981125593185